# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

## Backtesting Bullish Marubozu

In the previous notebook, we visualised bullish marubozu pattern. As a bullish marubozu indicates strong positive momentum. It is a good idea to take a long trade upon the formation of a bullish marubozu.

The strategy will be to enter a long trade if the previous candle is a bullish marubozu and exit the trade when either stop-loss or take-profit gets hit.

In this notebook, we will backtest the above trading strategy.

The notebook is structured as follows:
1. [Import the Data](#import)
2. [Entry Condition](#entry)
3. [Exit Condition](#exit)
3. [Backtesting](#backtesting)

## Import Libraries

In [1]:
# For data manipulation
import pandas as pd
import numpy as np

<a id='import'></a>
## Import the Data

Import the file `AAPL_marubozu.csv` using the `read_csv` method of `pandas`. This file contains the marubozu signals that we generated in the previous notebook.
This CSV file is available in the zip file of the unit 'Python Codes and Data' in the 'Course Summary' section.

In [2]:
# Import marubozu signal
data = pd.read_csv('../data_modules/AAPL_marubozu.csv', index_col=0)

# Change index to datetime
data.index = pd.to_datetime(data.index, format='%d-%m-%Y')

<a id='entry'></a>
## Entry Condition

We will check the conditions for entering the trade, set the `entry_signal` as `1` when `pattern_signal` is 100 . `1` indicates that we will initiate a long position. 

Entry: We will enter the trade at the `Open` of the current candle if the previous candle is a bullish marubozu.

In [3]:
# Generate entry signal
data['entry_signal'] = np.where(data['pattern_signal'] == 100, 1, 0)

<a id='exit'></a>
## Exit Condition

Stop-loss: The stop-loss will be kept at the minimum value of `Low` prices of the previous 3 candles including Marubozu. In the below example, we will keep the stop-loss at 40 as it is the low of the previous 3 candles.

Take-profit: We will book the profit if the price moves up by the same amount as the difference between stop-loss and entry price. In the below example, the entry price is 50 and the difference between entry price and stop-loss is 10 so we will keep the take-profit at 60.

We will define a function which takes dataframe `data`, `index`, `entry_price`, and number of previous candles `n` as inputs and returns `stoploss` and `take_profit` values.

![title](https://d2a032ejo53cab.cloudfront.net/Course/Units/Mcq/UnitContent/l5Xo7du9/SLTPbulluishmarubozu.png)

In [4]:
# Function to calculate SL and TP
def exit_values(df, index, entry_price, n):
    SL = data["Low"].iloc[index-n:index].min()
    TP = entry_price + (entry_price - SL) 
    return SL, TP

<a id='backtesting'></a>
## Backtesting

We will loop through each of the dates in the data and enter the long trade if the previous candle is a bullish marubozu. The trade will be exited when either the stop-loss or take-profit gets hit. 

We will backtest the strategy using the following steps:

**Step 1**: Create dataframe `trades` for storing trades.

**Step 2**: Initialise `current_position`, `trade_num` which is basically the number of trades, `cum_pnl` to 0 and set the `exit_flag` to `False`. 


In [5]:
# Create dataframe for storing trades
trades = pd.DataFrame(columns = ['Entry_Date', 'Entry_Price', 'Exit_Date', 'Exit_Type', 'Exit_Price'])

# Initialise current position, number of trades, cumulative pnl to 0
current_position = 0
trade_num = 0
cum_pnl = 0

# Set exit flag to False
exit_flag = False

Perform the following steps iteratively for the dates in the backtest period.

**Step 3**: For a given date if `data['entry_signal']` is `1` and `current_position` is `0` we will enter the trade.
1. Set `entry_date` as current date and `entry_price` as the opening price of the current candle.
2. Set the `stoploss` and `take_profit` using `exit_values` function. 
3. Update the `current_position` to 1 as now we have an open position.

**Step 4**: For a given date, if there is an open position, i.e. `current_position` is `1`.
1. Check for `stoploss` and `take_profit` conditions and set the `exit_flag` to `True` if any of the two conditions are met.
2. Check if `exit_flag` is `True`:<br>
  2.1. Set `exit_date` as current date and `exit_price` as the closing price of the current candle.<br>
  2.2. Calculate PnL for the trade which is basically the difference between `exit_price` and `entry_price`.<br>
  2.3. Append the trade to `trades` dataframe. This will be used in the future for strategy analysis.<br>
  2.4. Update `current_position` to `0` and set `exit_flag` to `False` as now we have exited the trade.

In [6]:
for index in range(len(data)):
    # Check if entry_signal is 1 and there is no open position (Step 3)
    if data['entry_signal'].iloc[index-1] == 1 and current_position == 0:
        
        # Set entry date and entry price (Step 3.1)
        entry_date = data.iloc[index].name
        entry_price = data['Open'].iloc[index]
        
        # Compute SL and TP for the trade (Step 3.2)
        stoploss, take_profit = exit_values(data, index, entry_price, 3)
        
        # Update current position to 1 (Step 3.3)
        current_position = 1
        
        # Increase number of trades by 1
        trade_num += 1   

        # Print trade details
        print("-"*30)
        print(f"Trade No: {trade_num} | Entry Date: {entry_date} | Entry Price: {entry_price}")           
     
    # Check if there is an open position of the given timestamp (Step 4)
    elif current_position == 1:
        
        # Exit the trade if any of the exit condition is met (Step 4.1)
        if data['Close'].iloc[index] < stoploss:
            exit_type = 'SL'
            exit_flag = True
            
        elif data['Close'].iloc[index] > take_profit:
            exit_type = 'TP'
            exit_flag = True         
 
        # Check if exit flag is true (Step 4.2)
        if exit_flag: 
            
            # Set exit date and exit price (Step 4.2.1)
            exit_date = data.iloc[index].name
            exit_price = data['Close'].iloc[index]
            
            # Calculate pnl for the trade (Step 4.2.2)
            trade_pnl = round(exit_price - entry_price,2)
            
            # Calculate cumulative pnl 
            cum_pnl = round(cum_pnl,2) + trade_pnl
            
            # Append the trade to trades dataframe (Step 4.2.3)
            trades = trades.append({'Entry_Date': entry_date, 'Entry_Price': entry_price, 
                                    'Exit_Date': exit_date, 'Exit_Type': exit_type, 
                                    'Exit_Price': exit_price, 'PnL': trade_pnl}, ignore_index=True)
            
            # Update current position to 0 and set exit flag to False (Step 4.2.4)
            current_position = 0
            exit_flag = False
            
            # Print trade details
            print(f"Trade No: {trade_num} | Exit Type: {exit_type} | Date: {exit_date} | Exit Price: {exit_price} | PnL: {trade_pnl} | Cum PnL: {round(cum_pnl,2)}")                              
            

------------------------------
Trade No: 1 | Entry Date: 2017-10-06 00:00:00 | Entry Price: 38.7425
Trade No: 1 | Exit Type: TP | Date: 2017-10-16 00:00:00 | Exit Price: 39.970001 | PnL: 1.23 | Cum PnL: 1.23
------------------------------
Trade No: 2 | Entry Date: 2018-01-11 00:00:00 | Entry Price: 43.647499
Trade No: 2 | Exit Type: TP | Date: 2018-01-12 00:00:00 | Exit Price: 44.272499 | PnL: 0.62 | Cum PnL: 1.85
------------------------------
Trade No: 3 | Entry Date: 2018-02-15 00:00:00 | Entry Price: 42.447498
Trade No: 3 | Exit Type: TP | Date: 2018-05-04 00:00:00 | Exit Price: 45.9575 | PnL: 3.51 | Cum PnL: 5.36
------------------------------
Trade No: 4 | Entry Date: 2018-05-11 00:00:00 | Entry Price: 47.372501
Trade No: 4 | Exit Type: SL | Date: 2018-06-25 00:00:00 | Exit Price: 45.5425 | PnL: -1.83 | Cum PnL: 3.53
------------------------------
Trade No: 5 | Entry Date: 2018-09-25 00:00:00 | Entry Price: 54.9375
Trade No: 5 | Exit Type: TP | Date: 2018-09-27 00:00:00 | Exit Pr

The above backtest results can be interpreted as follows:

The first trade was taken on `6th October, 2017`. The entry price was `38.75`. The trade was exited when the take-profit level was reached on `16th October, 2017`. The exit price was `39.97`. The PnL for the trade is `1.23` which is also the cumulative pnl as this is the first trade. 

We have generated `trades` dataframe which is basically a tradesheet. 

In [7]:
# Display the tradesheet
trades.head()

,Entry_Date,Entry_Price,Exit_Date,Exit_Type,Exit_Price,PnL
0,2017-10-06,38.742500,2017-10-16,TP,39.970001,1.23
1,2018-01-11,43.647499,2018-01-12,TP,44.272499,0.62
2,2018-02-15,42.447498,2018-05-04,TP,45.957500,3.51
3,2018-05-11,47.372501,2018-06-25,SL,45.542500,-1.83
4,2018-09-25,54.937500,2018-09-27,TP,56.237499,1.30


We will also create a column `trade_signal` containing the trading signals. It contains value `1` for all the rows when we had an open position and `0` when there was no open position. This will be used for analysing the performance of the strategy.

In [8]:
# Append signals as 1 and 0 on entry and exit dates
data.loc[data.index.isin(trades.Entry_Date),'trade_signal'] = 1
data.loc[data.index.isin(trades.Exit_Date),'trade_signal'] = 0

# Forward fill the NaN values
data['trade_signal'] = data['trade_signal'].ffill(axis = 0)

# Set the remaining NaN values to 0
data['trade_signal'] = data['trade_signal'].fillna(0)

# Display the data
data.tail()

,Open,High,Low,Close,pattern_signal,entry_signal,trade_signal
Date,,,,,,,
2022-09-12,159.589996,164.259995,159.300003,163.429993,0,0,0.0
2022-09-13,159.899994,160.539993,153.369995,153.839996,0,0,0.0
2022-09-14,154.789993,157.100006,153.610001,155.309998,0,0,0.0
2022-09-15,154.649994,155.240005,151.380005,152.369995,0,0,0.0
2022-09-16,151.210007,151.350006,148.369995,150.699997,0,0,0.0


## Conclusion

In this notebook, we saw how we can trade a bullish marubozu pattern with a defined stop-loss and take-profit level. We will  analyse the performance of our trading strategy so as to get a more profound idea about the strategy performance in the upcoming notebooks.